In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [2]:
def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the 
    code execution to a log file. Function returns nothing.'''
    
    timestamp_format = '%Y-%h-%d-%H:%M:%S' 
    now = datetime.now() 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [ ]:

def extract(url,tables_attributes):
    
    response = requests.get(url).text
    
    soup = BeautifulSoup(response, 'lxml')

    table = soup.find_all('tbody')[0]

    rows = table.find_all('tr')
    
    bank_names=[]
    market_caps=[]
    
    for row in rows[1:]:   
        col = row.find_all('td')
        if len(col) >= 3:
            bank_name = col[1].find_all('a')[1]['title']
            market_cap = float(col[2].contents[0][:-1])
            bank_names.append(bank_name)
            market_caps.append(market_cap)

    df=pd.DataFrame({tables_attributes[0]:bank_names,tables_attributes[1]:market_caps})
    df.rename(columns={tables_attributes[1]: 'MC_USD_Billion'}, inplace=True)
    return df

In [4]:
def transform(df, exchange_rate_csv):

    exchange_df = pd.read_csv(exchange_rate_csv)

    exchange_rate = exchange_df.set_index('Currency').to_dict()['Rate']

    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]

    return df



In [5]:
def load_to_csv(df,csv_path):
    df.to_csv(csv_path, index=False)



In [6]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    



In [7]:
def run_query(query_statement, sql_connection):
        query_output = pd.read_sql(query_statement, sql_connection)
        print(query_output)


In [10]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
tables_attributes = ['Bank_Name', 'Market_Cap_USD_Billion']
db_name = 'Banks.db'
table_name = 'Largest_banks'
exchange_rate_csv = './exchange_rate.csv'
csv_path = './Countries_by_GDP.csv'
log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url, tables_attributes)
log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df, exchange_rate_csv)
log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')
sql_connection = sqlite3.connect('World_Economies.db')
log_progress('SQL Connection initiated.')
load_to_db(df, sql_connection, table_name)
query_statement = f"SELECT * FROM Largest_banks"
log_progress(f'Data loaded to Database as table. Running the query {query_statement}')
run_query(query_statement, sql_connection)
query_statement = f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
log_progress(f'Data loaded to Database as table. Running the query {query_statement}')
run_query(query_statement, sql_connection)
query_statement = f"SELECT Bank_Name FROM Largest_banks LIMIT 5"
log_progress(f'Data loaded to Database as table. Running the query {query_statement}')
run_query(query_statement, sql_connection)
log_progress('Process Complete.')
sql_connection.close()

                                 Bank_Name  MC_USD_Billion
0                           JPMorgan Chase          432.92
1                          Bank of America          231.52
2  Industrial and Commercial Bank of China          194.56
3               Agricultural Bank of China          160.68
4                                HDFC Bank          157.91
5                              Wells Fargo          155.87
6                                     HSBC          148.90
7                           Morgan Stanley          140.83
8                  China Construction Bank          139.82
9                            Bank of China          136.81
                                 Bank_Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of Ch